In [14]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.polynomial.laguerre import laggauss
from numpy.polynomial.legendre import leggauss

# Gaussian Quadrature Notebook
This notebook implements the Gaussian Quadrature method from *Numerical Analysis* in multiple dimensions.

Gaussian Quadrature (below) is valid if $P(x)$ is any polynomial of degree less than $2n$. It is
$$
\int_{-1}^{1} P(x)\,dx = \sum_{i=1}^{n} c_i P(x_i)
$$
Otherwise, it will just give a decently good approximation.

In [15]:
### Define a few functions for testing

# 1-D
const_1D = lambda x: x**0
poly   = lambda x: 2*x**3 + 6*x**2 + 3*x + 6
expcos = lambda x: np.exp(x) * np.cos(x)
expsin = lambda x: x**6 - (x**2 * np.sin(2*x))
sin_1D = lambda x : np.sin(x)

# 2-D
const_2D = lambda x, y: (x*y)**0
ln_2d = lambda x, y: np.log(x + 2*y)
gaussian_2d = lambda x, y: np.exp(-x**2 -y**2)

# 3-D
const_3D = lambda x, y, z: (x*y*z)**0

In [35]:
### Gaussian Quadrature Implementation
def gaussQuad1D(f, a, b, n):
    """1-D Gaussian Quadrature using Legendre Polynomials"""
    roots, weights = leggauss(n)

    sum = 0
    # perform the summation
    for i in range(n):
        # get the roots into a,b from -1,1
        u = linTransform(roots[i], a, b)
        sum += weights[i] * f(u)

    # multiply by the coefficents from the variable substitution
    return 0.5 * (b-a) * sum

def linTransform(t, a, b):
    """Linear transform from [-1,1] to [a,b]"""
    return 0.5 * ((b - a) * t + (a + b))

In [33]:
print(f"T1: {gaussQuad1D(poly, -1, 1, 3):0.7f}")
print(f"T2: {gaussQuad1D(expcos, -1, 1, 3):0.7f}")
print(f"T3: {gaussQuad1D(expsin, 1, 3, 3):0.7f}")

#T1: 16.0000000
#T2: 1.9333905
#T3: 317.2641517


16.000000000000004
T1: 16.0000000
1.9333904692642978
T2: 1.9333905
317.264151733829
T3: 317.2641517


These match very well with the values provided in the book, so I'm confident in the routine.

## Gauss-Laguerre Quadrature
Figured I'd do this as a bonus- it does the following:
$$
\int_{0}^{\infty} e^{-x} f(x) \, dx \approx \sum_{i=1}^n c_i f(x_i)
$$

In [18]:
### This one inherently forces the bounds, so I don't think that any transformations will be necessary.
def gaussLaguerre1D(f, n):
    roots, weights = laggauss(n)

    sum = 0
    for i in range(n):
        sum += weights[i] * f(roots[i])

    return sum


print(f"G-L T1: {gaussLaguerre1D(const_1D, 6)}") # should be 1
print(f"G-L T2: {gaussLaguerre1D(sin_1D, 6)}")   # should be 0.5

G-L T1: 1.0000000000000002
G-L T2: 0.5000494747976744


## 2-D Gaussian Quadrature
The book provides the integral
$$
\int_{1.4}^{2.0} \int_{1.0}^{1.5} \ln (x+2y) \, dy \, dx \approx 0.4295545313.
$$

So, I'll base my initial construction on this thing. It looks to be defined as

$$
\int_{-1}^{1} \int_{-1}^{1} f(x, y) \, dy \, dx \approx \sum_{i=1}^{n} \sum_{j=1}^{n} c_i c_j P(x_i, x_j)
$$

In [19]:
def gaussQuad2D(f, x_bounds, y_bounds, n):
    """2-D Gaussian Quadrature using Legendre Polynomials"""
    # unpack variables needed for algo
    roots, weights = leggauss(n)
    a,b = x_bounds
    c,d = y_bounds

    # perform the double summation shown in the eq above
    sum = 0
    for i in range(n):
        # transform out of x
        u = linTransform(roots[i], a, b)
        for j in range(n):
            # transform out of y
            v = linTransform(roots[j], c, d)

            sum += weights[i] * weights[j] * f(u, v)

    # multiply by the extra coefficient needed from variable substitution
    return 0.25 * (b - a) * (d - c) * sum

In [20]:
print(gaussQuad2D(ln_2d, (1.4, 2.0), (1.0, 1.5), 3))
print(gaussQuad2D(gaussian_2d, (-1, 1), (-1, 1), 6))

nan
2.230983195257796


/tmp/ipykernel_55438/3898425507.py:12: RuntimeWarning: invalid value encountered in log
  ln_2d = lambda x, y: np.log(x + 2*y)


The first matches the value provided in the book, and the second agrees with the first few digits from WolframAlpha.

## 3-D Cartesian Coordinates Gaussian Quadrature
I'm just going to test this against a volume of a cube with side lengths = 2.
$$
\int_{0}^{2} \int_{0}^{2} \int_{0}^{2} 1 \, dz \, dy \, dx = 2^3 = 8
$$
I'll go out on a limb and say that the quadrature rule looks something like
$$
\int_{-1}^{1} \int_{-1}^{1} \int_{-1}^{1} f(x, y, z) \, dz \, dy \, dx \approx \sum_{i=1}^{n} \sum_{j=1}^{n} \sum_{k=1}^{n} c_i c_j c_k P(x_i, x_j, x_k)
$$

In [21]:
def gaussQuad3D(fxn, x_bounds, y_bounds, z_bounds, n):
    """3-D Gaussian Quadrature using Legendre Polynomials"""
    # unpack variables needed for algo
    roots, weights = leggauss(n)
    a,b = x_bounds
    c,d = y_bounds
    e,f = z_bounds

    # perform the triple(!) summation shown in the eq above
    sum = 0
    for i in range(n):
        # transform out of x
        u = linTransform(roots[i], a, b)

        for j in range(n):
            # transform out of y
            v = linTransform(roots[j], c, d)

            for k in range(n):
                #transform out of z
                w = linTransform(roots[k], e, f)
                sum += weights[i] * weights[j] * weights[k] * fxn(u, v, w)

    # multiply by the coefficients due to the variable transforms
    return 0.125 * (b-a) * (c - d) * (e - f) * sum

In [22]:
print(gaussQuad3D(const_3D, (0, 2), (0, 2), (0, 2), 1)) # should be 8

8.0


## 3-D Cylindrical Coordinates Gaussian Quadrature
I'll create this by using a simple test (that being the volume of a cylinder):
$$
\int_{0}^{2\pi} \int_{0}^{L} \int_{0}^{R} 1 r \, dr \, dz \, d\phi = \pi R^2 L
$$
I'll set R = 2, and L = 7 to obtain a volume of $28\pi \approx 87.964594$

In [23]:
def gaussQuad3DCyl(fxn, r_bounds, z_bounds, phi_bounds, n):
    """Performs 3-D Gaussian Quadrature on functions in cylindrical
    coordinates using Legendre Polynomials."""
    # multiply our function by the volume element
    cyl_fxn = lambda r, z, phi: fxn(r, z, phi) * r

    # unpack variables needed for algo
    roots, weights = leggauss(n) # these weighgivents are ones that go from -1 to 1
    a,b = r_bounds
    c,d = z_bounds
    e,f = phi_bounds

    # perform the triple(!) summation as done in the cartesian case
    sum = 0
    for i in range(n):
        # transform out of r
        u = linTransform(roots[i], a, b)

        for j in range(n):
            # transform out of z
            v = linTransform(roots[j], c, d)

            for k in range(n):
                #transform out of phi
                w = linTransform(roots[k], e, f)
                sum += weights[i] * weights[j] * weights[k] * cyl_fxn(u, v, w)

    # multiply by the coefficients due to the linear transforms
    return 0.125 * (b-a) * (c - d) * (e - f) * sum

In [24]:
gaussQuad3DCyl(const_3D, (0, 2), (0, 7), (0, 2*np.pi), 2)

np.float64(-87.9645943005142)

nice

## Gauss's Law Exercise
I'll solve Example 2.4 from Griffiths' *Introduction to Electrodynamics (4th Ed.)* using my quadrature methods.

### Analytical
The analytical solution for the Gauss's Law is
$$
|\textbf{E}| 2\pi s L = \frac{1}{\epsilon_0} \frac{2}{3} \pi k L r^3 \longrightarrow \textbf{E} = \frac{kr^2}{3\epsilon_0}  \hat{r}
$$

In [25]:
def efldCylinder(r, rho = None):
    """Evaluate the electric field at a radius `r away from the center of an
    infinite line of charge with charge density`rho`."""

    # if no rho provided, assume uniform charge density
    if rho is None:
        print("assume uniform")
        rho = lambda r, z, phi: 1

    # First, find q_enc/epsilon_0, but let epsilon_0 = 1 because its a hassle otherwise
    rhs = gaussQuad3DCyl(
        rho,
        r_bounds=(0, r),
        z_bounds=(0, 1),
        phi_bounds=(0, 2*np.pi),
        n=3
    )

    # Then, the electric field strength is just the enclosed charge divided by the volume
    # of the Gaussian cylinder
    return rhs / gaussQuad2D(
        lambda r,z: 2*np.pi,
        x_bounds=(0, r),
        y_bounds=(0, 1),
        n=2
    )

rho_example = lambda r, z, phi: 2.4 * r
print(f"Quadrature:{efldCylinder(5, rho_example)} \t Analytical {2.4 * 5**2 / 3}")

Quadrature:2.5280000000000014 	 Analytical 20.0


It works, but it's kind of ugly. My main gripe is with defining the functions for the quadrature implementations- I dont want to have to give extra parameters to the lambdas that I won't use just so it works.

In [26]:
import inspect

def lambdaHelper(fxn):
    
    print(f"This lambda depends on {inspect.signature(fxn).parameters}")
    
lambdaHelper(lambda z, r: r**2)


This lambda depends on OrderedDict({'z': <Parameter "z">, 'r': <Parameter "r">})
